# Arizona/California Weather Analysis

This is a report on the historical analysis of weather patterns in an area that approximately overlaps the area of Arizona and California (in fact, **SSSSSSBB** part).

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from this [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

We focus on six measurements:
* **TMIN, TMAX:** The daily minimum and maximum temperature (original data are in **tenths of degree Celsius**, here we convert them into degree Celsius).
* **TOBS:** The average temperature for each day (same metric as above).
* **PRCP:** Daily precipitation (in **tenths of mm**, sometimes we convert them into mm for comparison).
* **SNOW:** Daily snowfall (in **mm**).
* **SNWD:** The depth of accumulated snow (in **mm**).

## Sanity-check: Comparison with Outside Sources

We start by comparing some of the general statistics with graphs that we obtained from a site called [US Climate Data](http://www.usclimatedata.com/climate/boston/massachusetts/united-states/usma0046). Here for simplicity we **only** use statistics of **Phoenix in Arizona** to compare.

### TMIN, TMAX, TOBS

The graph below shows the daily minimum and maximum temperatures for each month, as well as the total precipitation for each month from US Climate Data.

<img src="figures/check-tmin-tmax.png" style="height:300px;" />

We see that the **shape of curve** of min and max daily temperatures agree with what we got from our data. Besides, **values** of TMAX match really well while **values** of TMIN from our data are a bit lower than US Climate Data. It might be due to the spots in California which have some slightly different values.

<img src="figures/real-tmin-tmax.png" style="height:300px;" />

Here the we can see the curve of TOBS has **similar shape** as TMIN and TMAX and the **values** lie between these two curves. The results match our expectation.
<img src="figures/real-tobs.png" style="height:300px;" />

### PRCP

We still use the figure from US Climate Data in last section to compare the precipitation:

<img src="figures/check-tmin-tmax.png" style="height:300px;" />

The following image shows approximately monthly precipitation in our data:

<img src="figures/real-prcp.png" style="height:300px;" />

To compare the precipitation, we translate tenths of mm/day to mm/month. From the figures we can find the **mean** of precipitation (PRCP) in our data matches really well with the statistics from US Climate Data. Actually, in both figures, there is **quite low precipitation** from April to June (**< 7 mm**). Besides, from September to November average precipitation is about **14 mm** and all other months have around **20 mm**.

### SNOW, SNWD

In fact, Phoenix in Arizona has **rarely** measureable snow fall in history. A [report](http://www.wrh.noaa.gov/psr/general/history/index.php?page=snow) shows that only **few** times there are measurable snow fall. In most days, the amount of snow fall is **0**.

In our data:
<img src="figures/real-snow-snwd.png" style="height:300px;" />
We can also see the **mean curve** (orange one) is almost zero. The standard deviation is much bigger than mean, which might be due to some far spots snowing sometimes. However, it is still low (**most < 5 mm**). These results match the statistics from the online report. In fact, these near-zero data have less value in further analysis, which will also be discussed later.

## PCA Analysis

For each of the six measurement, we compute the **percentage of the variance explained** as a function of the number of eigen-vectors used.

![](figures/var-tmin-tobs-tmax.png)

We see that the **top 10 eigenvectors** explain **36%** of variance for TMIN, **52%** for TOBS and **14%** for TMAX.

From the figures, we can conclude that of the three, TOBS is best explained by the top 10 eigenvectors. This is especially true for the first eigenvector which, by itself, explains **46%** of the variance. Besides, TOBS is an average value, which may have less noise than TMIN and TMAX.

Besides, typically the percentage of variance should be **monotonically increasing** w.r.t number of eigenvector considered. However, there are still some decreasing stages in all these 3 figures (eigenvector 2 in TMIN, TOBS and eigenvector 2,5,8 in TMAX). Those decreasing stages reflect **negative eigenvalues** of corresponding eigenvectors, which might be due to the missing data causing **the covariance matrix ill-formed**.

![](figures/var-snow-snwd-prcp.png)


The top 5 eigenvectors explain **20%** of the variance for PRCP, which is low. On the other hand the top 10 eigenvectors explain **73%** of the variance for SNOW and almost **100%** for SNWD. However, we still should **NOT** choose SNOW or SNWD for further analysis. It is because as we see in **1.1.3**, the mean of SNOW and SNWD is almost 0, which means most vectors have **near zero** coefficients for all eigenvectors and a **constant 0** is sufficient to express most vectors. There is not much need to analyze internal structure for the few non-zero vectors.

Besides, another proof can be seen in notebook for reconstruction of SNWD, where we filter out vectors for which the mean is a worse approximation than zero:
```
before filter 2282
after filter 23
```
It seems only few vectors left. Almost all vectors can be better expressed by 0.

To sum up, we will choose **TOBS** for deeper analysis.

## Analysis of Average Temperature (TOBS)

We choose to analyze the eigen-decomposition for average temperature because the first 4 eigen-vectors explain **45%** of the variance. Though not quite high, it is still the best choice currently. Besides, we also want to analyze the effect of eigenvector with negative eigenvalue (eigenvector 2).

### Mean and Eigenvectors

First, we graph the **mean** and the **top\* 4 eigenvectors** (in tenths of degree Celsius): <br>
(\*: eigenvector should not be listed in top eigenvectors, but we follow the result of program now)

<img src="figures/tobs-mean-eigs.png" style="height:300px;" />

We can find that the average temperature (TOBS) is almost **normally distributed**. 
The middle of **July** marks the peak of the average temperature and **January** is the coldest month.

Next we interpret the eigen-functions. The 1st eigen-function (eig1) has a shape similar to the mean function. One aspect of difference is that the eigen-function is higher in from June to September while in other months the curve is flat. The interpretation of this shape is that eig1 represents the **little extra temperature distinction** from June to September. Besides, we can see other than July and August, other months will have negative values, which means it can **globally decrease temperature**.

The 2nd eigen-function (eig2) has a quite similar shape of 1st eigen-function (eig1). But this time, values of all months are almost non-negative. It is like the shifted version of (eig1), which can represent the **extra temperature increase** from June to September. We can say (eig2) mainly make **summer hotter** but (eig1) mainly make **other seasons colder**.

The 3rd and 4th eigen-functions are kind of complementary in figure. The 3rd eigen-function can decrease temperature from March to April and increase temperature from October to January, which means a **colder spring** and a **warmer fall/winter**. The 4th eigen-function can decrease temperature from September to November and increase temperature from January to April, which means a **warmer winter/spring** and a **colder fall**.

###  Reconstruction Analysis

Here we use most positive and negative coefficients of certain eigenvectors to determine their effect on reconstruction.

#### Coeff1

Coeff1: most positive
![](figures/tobs-pos-coeff1.png)
Coeff1: most negative
![](figures/tobs-neg-coeff1.png)

As target curve shows, we cans see with most positive coeff1, the global average temperature is lower (temperature range changes from [100, 400] to [0, 300]), which match the **globally decrease temperature** expectation.

However, we do not observe much **extra temperature distinction** in summer (actually there is a little, as we can see the summer part in figures below is a bit flatter than the summer part above). It is due to the positive part in eigenvector 1 has low value, which may not contribute much (compared with noise).

#### Coeff2

Coeff2: most positive
![](figures/tobs-pos-coeff2.png)
Coeff2: most negative
![](figures/tobs-neg-coeff2.png)

Here we can see the figures with most positive coeff2 have **higher temperature in summer** (from around 250 to around 350), which matches our expection. Actually, we can see with most positive coeff2 almost **all months** have some increased temperature, which shows these figures may get both **positive effect of eigenvector 2** and **negative effect of eigenvector 1**, and this conclusion can be confirmed if we get both coeff1 and coeff2:

Coeff2: most positive
```
+-----------+------+------------------+------------------+------------------+
|    station|  year|           coeff_2|             res_2|           coeff_1|
+-----------+------+------------------+------------------+------------------+
|USC00028112|2002.0| 292.9100282706805|0.3232864604860019|-848.1323346363404|
|USC00022927|1966.0| 282.1602597567824|0.5924333947699456|-447.0810243046918|
|USC00028112|2004.0|265.03257181517876|0.3887194464613411|-835.8790377246141|
|USC00028112|1991.0| 264.8989014787868|0.4485906633669739|-685.6394504913202|
+-----------+------+------------------+------------------+------------------+
```

Coeff2: most negative
```
+-----------+------+-------------------+-------------------+------------------+
|    station|  year|            coeff_2|              res_2|           coeff_1|
+-----------+------+-------------------+-------------------+------------------+
|USC00022462|1955.0|-1871.2184473473887|0.41599489434169695|1253.0787636848488|
|USC00027460|1911.0| -1240.738500111339| 0.4494416936122674|1116.5204572787638|
|USC00022462|1956.0|-1176.8011449182054|0.26982025831051987|1292.1102875409135|
|USC00029287|1973.0|-1161.2975439348202| 0.3192334584439766|1633.4165014214734|
+-----------+------+-------------------+-------------------+------------------+
```

These results match our conclusion! **All most positive coeff2 have negative coeff1**. Besides, **all most negative coeff2 have positive coeff1**. The eigenvector 1 and eigenvector 2 cooperate together to increase/decrease the average temperature of the whole year.

#### Coeff3

Coeff3: most positive
![](figures/tobs-pos-coeff3.png)
Coeff3: most negative
![](figures/tobs-neg-coeff3.png)

Actually the eigenvector 3 oscillates a lot, which leads do some difficulty in analysis of its effect. Still, if we focus on TOBS in April, with most positive coeff3, the value (around 100) is lower than the one (around 150) with negative coeff3, which shows the effect of **colder spring**. However, the effect on months from October to January are not obvious (a really tiny bit around November). It is because the residual does not change much for the existence of eigenvector 3:


Coeff3: most positive
```
+-----------+------+-------------------+------------------+-------------------+
|    station|  year|              res_2|           coeff_3|              res_3|
+-----------+------+-------------------+------------------+-------------------+
|USC00045860|2010.0|  0.237312559403574| 409.8176596249946|0.20108238278605792|
|USC00022462|1959.0|0.17223272008850515|314.03938676736533| 0.1844530377649019|
|USC00022462|1983.0|0.36232996907284126|304.25147921855245|  0.304373939282402|
|USC00045860|2012.0|0.24844924072638191| 298.2044806751376|0.21830214915573282|
+-----------+------+-------------------+------------------+-------------------+
```

Coeff3: most negative
```
+-----------+------+-------------------+-------------------+-------------------+
|    station|  year|              res_2|            coeff_3|              res_3|
+-----------+------+-------------------+-------------------+-------------------+
|USC00020060|2000.0|0.23618197421804243|-372.49649231131355| 0.1924170254056361|
|USC00028499|1972.0|0.39748741902648965| -355.4961479987022| 0.2997216009189838|
|USC00029287|1956.0| 0.2875587045042562| -352.5201605871948|0.22882564242316794|
|USC00028489|1972.0| 0.3523433075167629|-317.03280777906514| 0.2710263922294451|
+-----------+------+-------------------+-------------------+-------------------+
```

In most positive coeff3, eigenvector 3 reduces 1% ~ 6% residual.
In most negative coeff3, eigenvector 3 reduces 4% ~ 10% residual.
Both are acceptable but not quite large, which may cause the change for the existence of eigenvector 3 not that obvious.

### Location Variation versus Year-to-year Variation on Coefficients

In the previous section we see the effects of eigenvectors with its coefficients on data points in average temperature. We now estimate the relative importance of location-to-location variation relative to year-by-year variation for different coefficient.

Firstly, we build tables of coefficients w.r.t. station and year. Then we compare the reduced fraction of variance when we subtract from each station/year entry the average-per-year or the average-per-station respectively. Here are the results:

** Coeff1 **  
```
total MS                    = 498922.307
MS removing mean-by-station = 239829.991, fraction explained = 51.9%
MS removing mean-by-year    = 420598.281, fraction explained = 15.7%
```

** Coeff2 **  
```
total MS                    = 250027.553
MS removing mean-by-station = 79785.341, fraction explained = 68.1%
MS removing mean-by-year    = 123157.961, fraction explained = 50.7%
```

** Coeff3 **  
```
total MS                    = 34814.772
MS removing mean-by-station = 31268.286, fraction explained = 10.2%
MS removing mean-by-year    = 15841.983, fraction explained = **54.5%**
```

We can see that in coeff1 and coeff2, the variation by station explains more than the variation by year. In fact, as we have seen in last section, these two often cooperate together to increase/decrease the average temperature **globally** on all days in the year, which can be seen as a station-based difference.

In coeff3, the variation by year explains much more than variation by station (55% vs. 10%), as eigenvector 3 has some **seasonal effect** which might cause the difference in years.

### Map-based Analysis

Here we will use a series of maps to show the relation between location parameters and the coefficients.

#### City Effect

Firstly we focus on stations around Phoenix and analyze the effect related to city.

<img src="figures/tobs-coeff1-map.png" style="height:400px;" />

In the map above, the area of circle represents the number of data points and the color shows the **coeff1** value. As we can see, most high coeff1 stations are in the center part of city (e.g. circles pointed by blue arrow) and most low coeff1 stations are in the outer part of city (e.g. circles pointed by red arrow). Due to the global temperature decrease effect of eigenvector 1, we may find **the center part has lower temperature** and **outer part has higher temperature**. It is an unexpected result that we often think the center part has more people and traffic, which may increase the temperature.

<img src="figures/tobs-coeff2-map.png" style="height:400px;" />

In the second map, the area of circle represents the number of data points and the color shows the **coeff2** value. This time the high coeff2 stations are around city (e.g. circles pointed by red arrow) and the low coeff2 stations are in the center of city (e.g. circles pointed by green arrow). Actually, the effect of eigenvector 2 is to make summer hotter. Thus, the **center part has colder summer** and the **outer part has hotter summer**, which match the result in first map.

#### Elevation Effect

Here we consider the relation between coefficient and elevation.

 <img src="figures/elevation-coeff1.png" style="height:300px;" /><img src="figures/elevation-coeff2.png" style="height:300px;" />

From the figure we can see, in fact the average coeff1 and coeff2 have **no obvious relation** with elevation of the station, though a lot of stations are built on elevation around 400. However, some conclusion can be drawn if we consider the topographical information. We will analyze it below.

#### Topographic Effect

<img src="figures/tobs-coeff1-topo.png" style="height:250px;" />

Though there is no obvious relation between coeff1 and elevation, if we draw the topographic figure, there is still some weak connection. In the figure, we can find the plain part (e.g. circles pointed by blue arrows) usually has lower coeff1, while the mountain part (e.g. circles pointed by yellow arrows) usually has higher coeff1. In fact it is explainable that **the high elevation of mountain part causes a lower average temperature** and **the low elevation of plain causes a higher average temperature**.

### Relation Between Two Stations

Here we follow the method in class to find a statistical test for rejecting the null hypothesis that says that the **average temperature in the two locations is independent**. Similarly we ignore the amount of average temperature and just ask whether it is high temperature (>a threshold T) in both locations. We can then compute the probability associated with the number of overlaps under the null hypothesis.

Fix two stations. We restrict our attention to the days for which we have measurements for both stations, and define the following notation:

* $m$ : the total number of days (for which we have measurements for both stations).
* $n_1$ : the number of days that it is above T on station 1
* $n_2$ : the number of days that it is above T on station 2
* $l$ : the number of days that it is above T on both stations.

We want to calculate the probability that the number of overlap days is $l$ given $m,n_1,n_2$.

The answer is:
$$
P = {m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} /{m \choose n_1}{m \choose n_2}
$$

Where
$$
{m \choose l,n_1-l,n_2-l,m-n_1-n_2+l} = \frac{m!}{l! (n_1-l)! (n_2-l)! (m-n_1-n_2+l)!}
$$

We use the fact that $\Gamma(n+1) = n!$ and denote $G(n) \doteq \log \Gamma(n+1)$
$$
\log P = \left[G(m) - G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) \right] - 
\left[G(m)-G(n_1)-G(m-n_1)\right] - \left[G(m)-G(n_2)-G(m-n_2)\right]
$$
Which slightly simplifies to 
$$
\log P = -G(l) -G(n_1-l) -G(n_2-l) -G(m-n_1-n_2+l) - G(m)+G(n_1)+G(m-n_1) +G(n_2)+G(m-n_2)
$$

The log probability scales with $m$ the length of the overlap (*Why?*). So to get a per-day significance we consider $
\frac{1}{m} \log P $

Here we compare 2 kinds of threshold (T=300 (30 degree Celsius), T=350 (35 degree Celsius)) to find correlation.

#### T=300

<img src="figures/tobs-matrix1-300.png" style="height:250px;" />

From the original P-value matrix, we can find the upper **left 20 stations** seem highly correlated.

<img src="figures/tobs-matrix2-300.png" style="height:450px;" />

Here we try to find more structures in the dependency matrix. After reordering the station by SVD, some internal correlations are more clear. From upper right matrix we can find **upper left 13 stations** are highly correlated and from lower left matrix we can find **upper left 8** stations are highly correlated.

#### T=350

<img src="figures/tobs-matrix1-350.png" style="height:250px;" />

If we change into a higher threshold, less stations will have experienced such high temperature. Thus, the color of P-value matrix is lighter, but we still can find the **upper left 20 stations** seem more correlated.

<img src="figures/tobs-matrix2-350.png" style="height:450px;" />

However, if we use SVD to reorder the stations, it seems about **13 stations** (yellow ones) have almost no correlation with other stations, which means these stations may have very few days above 35 degree Celsius. From lower left matrix we can see upper left **8 stations** and lower right **8 stations** are highly correlated, but actually there are not many internal structures.

#### Map for T=350

<img src="figures/tobs-cor-map.png" style="height:250px;" />

If we draw the first 4 coefficients graph, there are some local structures: in green rectangle the negative green triangle is bigger, in blue rectangle the negative blue triangle is somewhat bigger and in orange rectangle the positive orange triangle is bigger. These triangles' area is corresponding to the coefficients of 4 eigenvectors. However, there are **no many obvious structures** for those eigenvectors. 